# **Heston Model Calibration**

##**1. Necessary Libraries**

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# from google.colab import drive
# drive.mount('/content/drive')

# Calculation and Optimization
import scipy.stats as ss
import scipy.optimize as scpo
from scipy import sparse
from scipy.fftpack import ifft
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from scipy.optimize import minimize
from functools import partial
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# Visualization
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import warnings; warnings.filterwarnings('ignore')

##**2. Models**

Notation:

* $S_0$: current stock price
* $K$: strike price
* $T$: time to maturity
* $v_0$: spot variance
* $r$: risk-free rate (Treasury Bill Yield)
* $\mu$: constant drift
* $\rho$: correlation between the stock price process and variance process
* $\theta$: longterm mean of the variance process
* $\gamma$: volatility coefficient of the variance process
* $\kappa$: mean reversion coefficient for the variance process
* $N$: Number of expansion terms in COS method
* $L$: size of truncation domain of COS method (typ.:L=8 or L=10)  


### **2.1 Process Initialization**

In [ ]:
class Heston_process():
    def __init__(self, r = 0.035, rho = 0, gamma = 0, theta = -0.1, kappa = 0.1):
        self.r, self.rho, self.theta, self.gamma, self.kappa = r, rho, theta, gamma, kappa

In [ ]:
class Option_param():
    def __init__(self, S0, K, T, v0 = 0.05,
               payoff = 'call', exercise = 'European'):
        self.S0, self.v0, self.K, self.T, self.exercise, self.payoff = S0, v0, K, T, exercise, payoff

### **2.2 Numerical Method - COS**

In [ ]:
def truncationRange(L, r, T, v0, theta, kappa, rho, gamma):
    c1 = r * T + (1 - np.exp(-kappa * T)) * (theta - v0)/(2 * kappa) - theta * T / 2


    c2 = 1/(8 * np.power(kappa,3)) * (gamma * T * kappa * np.exp(-kappa * T) \
        * (v0 - theta) * (8 * kappa * rho - 4 * gamma) \
        + kappa * rho * gamma * (1 - np.exp(-kappa * T)) * (16 * theta - 8 * gamma) \
        + 2 * theta * kappa * T * (-4 * kappa * rho * gamma + np.power(gamma,2) + 4 * np.power(kappa,2)) \
        + np.power(gamma,2) * ((theta - 2 * v0) * np.exp(-2 * kappa * T) \
        + theta * (6 * np.exp(-kappa * T) - 7) + 2 * v0) \
        + 8 * np.power(kappa,2) * (v0 - theta) * (1 - np.exp(-kappa * T)))

    a = c1 - L * np.sqrt(np.abs(c2))
    b = c1 + L * np.sqrt(np.abs(c2))
    return a, b

In [ ]:
def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c

    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0))
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi *
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)

    value = {"chi":chi,"psi":psi }
    return value

def CallPutCoefficients(a,b,k):
    c = 0.0
    d = b
    coef = Chi_Psi(a,b,c,d,k)
    Chi_k = coef["chi"]
    Psi_k = coef["psi"]
    if a < b and b < 0.0:
        H_k = np.zeros([len(k),1])
    else:
        H_k = 2.0 / (b - a) * (Chi_k - Psi_k)
    return H_k

def COS_method(cf, S0, r, T, K, N, L, aTruncated, bTruncated):

    # reshape K to a column vector
    if K is not np.array:
        K = np.array(K).reshape([len(K),1])

    #assigning i=sqrt(-1)
    i = np.complex(0.0,1.0)
    x0 = np.log(S0/K)

    # # truncation domain
    # a = 0.0 - L * np.sqrt(T)
    # b = 0.0 + L * np.sqrt(T)
    a = aTruncated
    b = bTruncated
    # a, b = truncationRange()


    # sumation from k = 0 to k=N-1
    k = np.linspace(0,N-1,N).reshape([N,1])
    u = k * np.pi / (b - a);

    # Determine coefficients for Put Prices
    H_k = CallPutCoefficients(a,b,k)
    mat = np.exp(i * np.outer((x0 - a) , u))
    temp = cf(u) * H_k
    temp[0] = 0.5 * temp[0]
    value = np.exp(-r * T) * K * np.real(mat.dot(temp))
    return value

### **2.3 Option Pricing**

Characteristic function of Heston model for the log-asset price
$$\phi(u;v_0) = \exp(iurT+\dfrac{v_0}{\gamma^2}(\dfrac{1-e^{-DT}}{1-Ge^{-DT}})(\kappa-i\rho\gamma u -D)+\dfrac{\kappa \theta}{\gamma^2}(T(\kappa-i\rho\gamma u -D)-2\log(\dfrac{1-Ge^{-DT}}{1-G})))$$

where $D=\sqrt{(\kappa-i\rho\gamma u)^2+(u^2+iu)\gamma^2}$, and

$G = \dfrac{\kappa-i\rho\gamma u-D}{\kappa-i\rho\gamma u+D}$

To determine the interval of integration $[a,b]$ applying for COS method, we define the truncation range as:
$$[a,b]=[\epsilon_1-L\sqrt{\epsilon_2},\epsilon_1+L\sqrt{\epsilon_2}]$$
where: $\epsilon_n$ is the n-th cumulant of $ln(S_T/K)$
$$\epsilon_n = \dfrac{1}{i^n}\dfrac{\partial^n(t\phi(\omega))}{\partial\omega^n}$$

In [ ]:
class Heston_pricer():
    def __init__(self, Option_params, Process_params):
        """
        Process_params: a instance of "Heston_process.", which contains (r, rho, gamma, theta, kappa)
        Option_params: of type Option_param, which contains (S0,K,T)
        """

        self.r = Process_params.r
        self.rho = Process_params.rho
        self.theta = Process_params.theta
        self.gamma = Process_params.gamma
        self.kappa = Process_params.kappa

        self.S0 = Option_params.S0
        self.v0 = Option_params.v0
        self.K = Option_params.K
        self.T = Option_params.T
        self.exercise = Option_params.exercise
        self.payoff = Option_params.payoff

    def payoff_Func(self, S):
        if self.payoff == "call":
            Payoff = np.maximum( S - self.K, 0 )
        elif self.payoff == "put":
            Payoff = np.maximum( self.K - S, 0 )
        return Payoff

    def COS(self, K):
        K = np.array(K)

        # Characteristic function of Heston model for log asset price
        def cf_Heston(u,r,T,kappa,gamma,theta,v0,rho):
            i = np.complex(0.0,1.0)
            D1 = np.sqrt(np.power(kappa-gamma*rho*i*u,2)+(u*u+i*u)*gamma*gamma)
            g = (kappa-gamma*rho*i*u-D1)/(kappa-gamma*rho*i*u+D1)
            C = (1.0-np.exp(-D1*T))/(gamma*gamma*(1.0-g*np.exp(-D1*T)))\
                *(kappa-gamma*rho*i*u-D1)
            A = r * i*u * T + kappa*theta*T/gamma/gamma *(kappa-gamma*rho*i*u-D1)\
                - 2*kappa*theta/gamma/gamma*np.log((1.0-g*np.exp(-D1*T))/(1.0-g))

            cf = np.exp(A + C*v0)
            return cf

        def truncationRange(L, r, T, rho, gamma, theta, kappa, v0):
            c1 = r * T + (1 - np.exp(-kappa * T)) * (theta - v0)/(2 * kappa) - theta * T / 2

            c2 = 1/(8 * np.power(kappa,3)) * (gamma * T * kappa * np.exp(-kappa * T) \
                * (v0 - theta) * (8 * kappa * rho - 4 * gamma) \
                + kappa * rho * gamma * (1 - np.exp(-kappa * T)) * (16 * theta - 8 * gamma) \
                + 2 * theta * kappa * T * (-4 * kappa * rho * gamma + np.power(gamma,2) + 4 * np.power(kappa,2)) \
                + np.power(gamma,2) * ((theta - 2 * v0) * np.exp(-2 * kappa * T) \
                + theta * (6 * np.exp(-kappa * T) - 7) + 2 * v0) \
                + 8 * np.power(kappa,2) * (v0 - theta) * (1 - np.exp(-kappa * T)))

            a = c1 - L * np.sqrt(np.abs(c2))
            b = c1 + L * np.sqrt(np.abs(c2))
            return a, b

        aTruncated, bTruncated = truncationRange(10, self.r, self.T, self.rho,
                                             self.gamma, self.theta, self.kappa, self.v0)

        cf = partial(cf_Heston, T=self.T, v0=self.v0, r=self.r, theta=self.theta,
                                      gamma=self.gamma, kappa=self.kappa, rho=self.rho)

        return COS_method(cf, self.S0, self.r, self.T, K, 1000, 10, aTruncated, bTruncated)

In [ ]:
def report_calibration(initial_guess, calibrated_params):
    report = pd.DataFrame({"Initial guess": initial_guess, "Calibrated": calibrated_params},
                          index=["rho", "gamma", "theta", "kappa", "v0"]).round(4).T
    return report

## **3. Data Implementation**

## **3.1. Data description**

Data used for this project is the end-of-date (EOD) call option chain data of S&P 500 index option (SPX), which belongs to European option type. The extracted data of the year 2022.

Source: [SPX Option Chain - optionsDX](https://www.optionsdx.com/product/spx-option-chain/)

In [ ]:
df = pd.read_csv('/content/spx_eod_202302.csv')


FileNotFoundError: ignored

In [ ]:
def weight_error(metric):
    if metric == 'rmse':

        return wrmse

    if metric == 'mape':

        return wmape



In [ ]:
df[['[STRIKE]','[C_VOLUME]']].iloc[0:92,:]

,[STRIKE],[C_VOLUME]
189,4745.0,267.0
191,4755.0,482.0
199,4795.0,12513.0
200,4800.0,21178.0
201,4805.0,11909.0
...,...,...
465,4775.0,1634.0
466,4780.0,573.0
467,4785.0,352.0
468,4790.0,2029.0


In [ ]:
data = pd.read_csv('/content/drive/My Drive/WORK/IU/Research/data.csv')
data.iloc[0:92,:]

,[UNDERLYING_LAST],[STRIKE],[C_BID],[C_ASK],[DTE],[C_MIDPOINT],[SPREAD],Heston-COS Call Price
0,4795.57,4320.0,473.90,478.10,2.0,476.00,-4.2,476.821368
1,4795.57,4345.0,448.90,453.10,2.0,451.00,-4.2,451.420191
2,4795.57,4350.0,443.90,448.10,2.0,446.00,-4.2,446.523356
3,4795.57,4375.0,418.90,423.10,2.0,421.00,-4.2,422.441976
4,4795.57,4380.0,413.90,418.10,2.0,416.00,-4.2,417.550240
...,...,...,...,...,...,...,...,...
87,4795.57,4860.0,0.35,0.45,2.0,0.40,-0.1,1.041782
88,4795.57,4865.0,0.25,0.35,2.0,0.30,-0.1,0.972546
89,4795.57,4870.0,0.15,0.25,2.0,0.20,-0.1,0.882527
90,4795.57,4875.0,0.10,0.20,2.0,0.15,-0.1,0.770333


In [ ]:
data = pd.read_csv('/content/drive/My Drive/WORK/IU/Research/spx_eod.csv')
df = pd.DataFrame(data)
df = df.dropna()

In [ ]:
# Intepret the data, by individual options - that is, by expire date and strike
df1 = df.groupby(by = '[EXPIRE_DATE]')
df2 = {}
df21 = pd.DataFrame()

for expiry in df['[EXPIRE_DATE]'].unique():
    df2[expiry] = df1.get_group(expiry)
    df21 = df21.append(df2[expiry])

df21 = df21.set_index(['[EXPIRE_DATE]','[QUOTE_DATE]'], drop = False)

In [ ]:
# Define dataset to for studying the model
data = df21[['[UNDERLYING_LAST]','[STRIKE]','[C_BID]','[C_ASK]','[DTE]']]

data['[C_MIDPOINT]'] = (data['[C_BID]'] + data['[C_ASK]'])/2
data['[SPREAD]'] = (data['[C_BID]'] - data['[C_ASK]'])

# Remove data which may cause ...
data = data[data['[DTE]']!=0.0]
data = data[data['[SPREAD]'] != 0]

In [ ]:
# Close price of SPX
stockData = df.groupby(by = '[QUOTE_DATE]').first()
stockData = pd.DataFrame(stockData['[UNDERLYING_LAST]'])

# Return and Volatility
logReturn = np.log(stockData['[UNDERLYING_LAST]']) - np.log(stockData['[UNDERLYING_LAST]'].shift(1))
logReturn.drop(logReturn.index[:1], inplace = True)
# tradingDaysCount   = 252
annualisedMean     = np.mean(logReturn) * 252
annualisedVariance = np.var(logReturn) * 252
annualisedStdDev   = np.sqrt(annualisedVariance)

# pred = pd.DataFrame()
# pred = pd.read_csv('/content/drive/My Drive/WORK/IU/Research/pred.csv')
# pred = pd.DataFrame(pred)

In [ ]:
p =pd.DataFrame()
p1 =[[1,2,3,4]]
p2=[[2,3,4,5]]
p= p.append(p1)
p=p.append(p2)
p

,0,1,2,3
0,1,2,3,4
0,2,3,4,5


## **3.2. Implementing the model - Results**

In [ ]:
r=0.035
i=0
pred = pd.DataFrame()
params_Hest_df = pd.DataFrame()

for date in data.index.unique().values:
    S0 = data.loc[date]['[UNDERLYING_LAST]'][0]
    strikes = data.loc[date]['[STRIKE]'].values
    prices = data.loc[date]['[C_MIDPOINT]'].values
    spreads = data.loc[date]['[SPREAD]'].values
    T = data.loc[date]['[DTE]'][0]/365

    # Objective function
    def f_Hest(x, rho, gamma, theta, kappa, v0, r=0.035):
        Heston_params = Heston_process(r=r, rho=rho, gamma=gamma, theta=theta, kappa=kappa)
        Option_params = Option_param(S0=S0, K=S0, T=T, v0=v0, exercise="European", payoff='call')
        Hest = Heston_pricer(Option_params, Heston_params)
        return Hest.COS(x).flatten()

    init_vals = [-0.5, 10.0, 0.01, 10.0, 0.2] # rho, gamma, theta, kappa, v0
    bounds = ([-1, 1e-15, 1e-15, 1e-15, 1e-15], [1, np.inf, 2, np.inf, 2] )
    # return the calibrated set of 5 params
    params_Hest = scpo.curve_fit(f_Hest, xdata = strikes, ydata = prices,
                                p0 = init_vals, bounds = bounds, sigma = spreads,
                                xtol = 1e-4, max_nfev = 2000)[0]
    params_Hest_df = params_Hest_df.append([params_Hest])

    # def Feller(x):
    #     return 2*x[3] * x[2] - x[1]**2 - 1e-6
    # cons = ({"fun": Feller, "type": "ineq"})

    # def least_sq(x, prices, strikes, spread):
    #     """ Objective function """
    #     Heston_param = Heston_process(mu=r, rho=x[0], sigma=x[1], theta=x[2], kappa=x[3])
    #     opt_param = Option_param(S0=S0, K=K, T=T, v0=x[4], exercise="European", payoff="call" )
    #     Hest = Heston_pricer(opt_param, Heston_param)
    #     prices_calib = Hest.FFT(strikes)
    #     return np.sum( ((prices_calib - prices)/spread)**2 )

    # init_vals = [-0.4, 1.1, 0.1, 0.6, 0.02] # rho, sigma, theta, kappa, v0
    # bounds = ( (-1,1), (1e-15,np.inf), (1e-15, 50), (1e-15, 50), (1e-15, 10) )
    # params_Hest_con = scpo.minimize(least_sq, x0=init_vals, args=(prices, strikes, spreads),
    #                   method='SLSQP', bounds=bounds,
    #                   constraints=cons, tol=1e-4, options={"maxiter":500}).x

    print(i)
    print(date,params_Hest)

    i += 1
    P = pd.DataFrame(f_Hest(strikes, params_Hest[0], params_Hest[1], params_Hest[2],
                       params_Hest[3], params_Hest[4], r = r))
    pred = pd.concat([pred, P])
# 8:13
# Result
# report_calibration(init_vals, params_Hest)

NameError: ignored

In [ ]:
# data['Heston-COS Call Price'] = pred
data.to_csv('/content/drive/My Drive/WORK/IU/Research/data.csv',
          encoding='utf-8', index=False)

In [ ]:

data

[UNDERLYING_LAST]  [STRIKE]  [C_BID]  [C_ASK]  \
[EXPIRE_DATE] [QUOTE_DATE]                                                  
2022-01-05    2022-01-03              4795.57    4320.0    473.9    478.1   
              2022-01-03              4795.57    4345.0    448.9    453.1   
              2022-01-03              4795.57    4350.0    443.9    448.1   
              2022-01-03              4795.57    4375.0    418.9    423.1   
              2022-01-03              4795.57    4380.0    413.9    418.1   
...                                       ...       ...      ...      ...   
2023-02-24    2022-12-29              3848.66    3810.0    161.1    161.9   
              2022-12-30              3839.81    3750.0    190.5    193.9   
              2022-12-30              3839.81    3810.0    152.9    153.8   
              2022-12-30              3839.81    3830.0    136.7    144.0   
2023-02-01    2022-12-30              3839.81    3800.0    122.9    126.5   

                            [DTE]  [C_MIDPOINT]  [SPREAD]  \
[EXPIRE_DATE] [QUOTE_DATE]                                  
2022-01-05    2022-01-03      2.0        476.00      -4.2   
              2022-01-03      2.0        451.00      -4.2   
              2022-01-03      2.0        446.00      -4.2   
              2022-01-03      2.0        421.00      -4.2   
              2022-01-03      2.0        416.00      -4.2   
...                           ...           ...       ...   
2023-02-24    2022-12-29     57.0        161.50      -0.8   
              2022-12-30     56.0        192.20      -3.4   
              2022-12-30     56.0        153.35      -0.9   
              2022-12-30     56.0        140.35      -7.3   
2023-02-01    2022-12-30     33.0        124.70      -3.6   

                            Heston-COS Call Price  
[EXPIRE_DATE] [QUOTE_DATE]                         
2022-01-05    2022-01-03               476.821368  
              2022-01-03               451.420191  
              2022-01-03               446.523356  
              2022-01-03               422.441976  
              2022-01-03               417.550240  
...                                           ...  
2023-02-24    2022-12-29               161.500995  
              2022-12-30               192.509093  
              2022-12-30               153.298252  
              2022-12-30               141.897301  
2023-02-01    2022-12-30               124.700000  

[842149 rows x 8 columns]

In [ ]:
from sklearn.metrics import mean_absolute_error

print(sqrt(mean_squared_error(data['[C_MIDPOINT]'], data['Heston-COS Call Price']))

11.983628659069682


In [ ]:
print(mean_absolute_percentage_error(data['[C_MIDPOINT]'], data['Heston-COS Call Price']))

0.06045244017878366


In [ ]:
data['Heston-COS Call Price'] = pred.values.flatten()

ValueError: ignored

In [ ]:
S0 = data.loc['2022-01-05', '2022-01-03']['[UNDERLYING_LAST]'][0]
strikes = data.loc['2022-01-05', '2022-01-03']['[STRIKE]'].values
prices = data.loc['2022-01-05', '2022-01-03']['[C_MIDPOINT]'].values
spreads = data.loc['2022-01-05', '2022-01-03']['[SPREAD]'].values
T = data.loc['2022-01-05', '2022-01-03']['[DTE]'][0]/365
r=0.035


# Objective function
def f_Hest(x, rho, gamma, theta, kappa, v0, r=0.035):
    Heston_params = Heston_process(r=r, rho=rho, gamma=gamma, theta=theta, kappa=kappa)
    Option_params = Option_param(S0=S0, K=S0, T=T, v0=v0, exercise="European", payoff='call')
    Hest = Heston_pricer(Option_params, Heston_params)
    return Hest.COS(x).flatten()

init_vals = [0, 10.0, 0.0001, 10.0, 0.2] # rho, gamma, theta, kappa, v0
bounds = ([-1, 1e-15, 1e-15, 1e-15, 1e-15], [1, np.inf, 2, np.inf, 2] )
#
params_Hest = scpo.curve_fit(f_Hest, xdata = strikes, ydata = prices,
                             p0 = init_vals, bounds = bounds, sigma = spreads,
                             xtol = 1e-4, max_nfev = 1000)[0]

# Result
report_calibration(init_vals, params_Hest)

,rho,gamma,theta,kappa,v0
Initial guess,0.0000,10.0000,0.0001,10.0000,0.2
Calibrated,-0.7652,19.7325,2.0000,5.8054,0.0


In [ ]:
a = pd.DataFrame(f_Hest(strikes, params_Hest[0], params_Hest[1], params_Hest[2],
                       params_Hest[3], params_Hest[4], r = r))

In [ ]:
a, b = truncationRange(L=10, r=0.035, T=T, v0=params_Hest[4],
                       theta=params_Hest[2], kappa=params_Hest[3],
                       rho=params_Hest[0], gamma=params_Hest[1])

In [ ]:
result = pd.DataFrame(f_Hest(strikes, params_Hest[0], params_Hest[1],
                             params_Hest[2], params_Hest[3],
                             params_Hest[4], r = 0.035), index = strikes)

result['real'] = data.loc['2022-01-05', '2022-01-03']['[C_MIDPOINT]'].values

In [ ]:
result

,0,real
4320.0,476.829460,476.000
4345.0,451.418796,451.000
4350.0,446.519045,446.000
4375.0,422.435097,421.000
4380.0,417.545576,416.000
...,...,...
4865.0,0.971878,0.300
4870.0,0.882082,0.200
4875.0,0.770124,0.150
4885.0,0.506240,0.100


In [ ]:
ITM_result = result.iloc[0:30,]
ATM_result = result.iloc[31:61,]
OTM_result = result.iloc[62:92,]

print(np.sqrt(mean_squared_error(ITM_result['real'], ITM_result[0]))/S0*100)
print(np.sqrt(mean_squared_error(ATM_result['real'], ATM_result[0]))/S0*100)
print(np.sqrt(mean_squared_error(OTM_result['real'], OTM_result[0]))/S0*100)

0.03515767969956777
0.08801348814956929
0.06154732940155872
